# Converte CSV para Padrão IOB

## Load csv with text and IOB

In [8]:
#!virtualenv --python=python3.8 venv

!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113

!pip install seqeval

!pip install scikit-learn==0.24

!pip install sklearn==0.0

!pip install sklearn-crfsuite==0.3.6

!pip install python-crfsuite==0.9.7

!pip install tqdm

!pip install pytorch-crf==0.7.2

#!pip install torch==1.8.1

#!pip install torch==1.11.0

!pip install torch-summary

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113


In [2]:
import pandas as pd

import nltk

import scipy.stats

import sklearn
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

#import matplotlib.pyplot as plt
#import seaborn as sns
#sns.set(color_codes=True)
#sns.set(font_scale=1)
#%matplotlib inline
#%config InlineBackend.figure_format = 'svg'


In [9]:
data = pd.read_csv('/home/lucelia_vieira/Experimentos/ner/CSVs/IOBabertura.csv')
#data = pd.read_csv('/home/lucelia_vieira/Experimentos/ner/CSVs/DODFCorpus_contratos_licitacoes_v0.csv')

In [13]:
labels = data["tipo_ent"].unique()

In [7]:
tipo = data["tipo_rel"].unique()
tipo

array(['REL_EXTRATO_CONTRATO', 'REL_ADITAMENTO_CONTRATO',
       'REL_AVISO_LICITACAO', 'REL_ANUL_REVOG_LICITACAO',
       'REL_SUSPENSAO_LICITACAO', 'REL_EXTRATO_CONVENIO'], dtype=object)

In [18]:
labels.sort()
labels

array(['AVISO_ANUL_REV_LICITACAO', 'AVISO_LICITACAO',
       'AVISO_SUSPENSAO_LICITACAO', 'EXTRATO_ADITAMENTO_CONTRATUAL',
       'EXTRATO_CONTRATO', 'EXTRATO_CONVENIO', 'cnpj_entidade_contratada',
       'cnpj_entidade_convenente', 'cnpj_orgao_concedente',
       'cnpj_orgao_contratante', 'codigo_licitacao_sistema_compras',
       'data_abertura_licitacao', 'data_assinatura_contrato',
       'data_assinatura_convenio', 'data_escrito', 'decisao_tcdf',
       'entidade_contratada', 'entidade_convenente', 'fonte_recurso',
       'identificacao_ocorrencia', 'modalidade_licitacao',
       'natureza_despesa', 'nome_responsavel', 'nota_empenho',
       'numero_contrato', 'numero_convenio', 'numero_licitacao',
       'numero_termo_aditivo', 'objeto_aditamento_contratual',
       'objeto_contrato', 'objeto_convenio', 'objeto_licitacao',
       'orgao_concedente', 'orgao_contratante', 'orgao_licitante',
       'processo_gdf', 'programa_trabalho', 'sistema_compras',
       'tipo_objeto', 'unidad

In [16]:
data

,Unnamed: 0,ATO,DATA,PROCESSO,TIPO,DISPUTA,V_ESTIMADO,REDATOR,Texto,IOB
0,0,AVISO DE ABERTURA DE LICITACAO,23 de janeiro de 2004,NaN,NaN,NaN,NaN,NaN,AVISO DE ABERTURA DE LICITACAO TOMADA DE PRECO...,B-ATO I-ATO I-ATO I-ATO I-ATO O O O O O O O O ...
1,1,AVISO DE ABERTURA DE LICITACAO,29 de janeiro de 2004,NaN,NaN,NaN,NaN,NaN,AVISO DE ABERTURA DE LICITACAO TOMADA DE PRECO...,B-ATO I-ATO I-ATO I-ATO I-ATO O O O O O O O O ...
2,2,AVISO DE ABERTURA DE LICITACAO,04-02-2004,1.018682e+09,NaN,NaN,NaN,NaN,AVISO DE ABERTURA DE LICITACAO PREGAO No 001/2...,B-ATO I-ATO I-ATO I-ATO I-ATO O O O O O O O O ...
3,3,AVISO DE ABERTURA DE LICITACAO,25-02-2004,1.001312e+10,NaN,NaN,NaN,NaN,AVISO DE ABERTURA DE LICITACAO CONVITE No 001/...,B-ATO I-ATO I-ATO I-ATO I-ATO O O O O O O O O ...
4,4,AVISO DE ABERTURA DE LICITACAO,25-02-2004,1.001312e+10,NaN,NaN,NaN,NaN,AVISO DE ABERTURA DE LICITACAO CONVITE No 001/...,B-ATO I-ATO I-ATO I-ATO I-ATO O O O O O O O O ...
...,...,...,...,...,...,...,...,...,...,...
1238,1238,AVISO DE ABERTURA DE LICITACAO,24 de agosto de 2021,5.000035e+15,Menor Preco,Aberto,"R$ 3.864.574,44",AMILCAR UBIRATAN URACH VIEIRA,AVISO DE ABERTURA DE LICITACAO PREGAO ELETRONI...,B-ATO I-ATO I-ATO I-ATO I-ATO O O O O O O B-PR...
1239,1239,AVISO DE ABERTURA DE LICITACAO,25-08-2021,7.200001e+15,Menor Preco por item,NaN,NaN,JOAO DE DEUS ABREU SOARES,AVISO DE ABERTURA DE LICITACAO PREGAO ELETRONI...,B-ATO I-ATO I-ATO I-ATO I-ATO O O O O O O B-PR...
1240,1240,AVISO DE ABERTURA DE LICITACAO,26-08-2021,4.026000e+17,Menor Preco,NaN,"R$ 19.940,00",ROSIMEIRE PAIVA DA SILVA,AVISO DE ABERTURA DE LICITACAO SRP PREGAO ELET...,B-ATO I-ATO I-ATO I-ATO I-ATO O O O O O O O B-...
1241,1241,AVISO DE ABERTURA DE LICITACAO,26-08-2021,7.200000e+15,Menor Preco por Grupo,NaN,NaN,JOAO DE DEUS ABREU SOARES,AVISO DE ABERTURA DE LICITACAO PREGAO ELETRONI...,B-ATO I-ATO I-ATO I-ATO I-ATO O O O O O O B-PR...


In [10]:
#extrai do csv a coluna texto e tags IOB
DEFAULT_TOKENIZER = nltk.RegexpTokenizer(r"\w+").tokenize
x = []
y = []
for row in range(len(data)):
    if pd.notna(data['Texto'][row]) and pd.notna(data['IOB'][row]):
        x.append(DEFAULT_TOKENIZER(data['Texto'][row]))
        y.append(data['IOB'][row].split())
len(x), len(y)

(1243, 1243)

In [28]:
z = ''
for row in range(len(data)) :
    if pd.notna(data['Texto'][row]) and pd.notna(data['IOB'][row]):
        # concatenating the columns
        z = data['Texto'].map(str) + ' ' + data['IOB'].map(str)
        print(z)
with open('/home/lucelia_vieira/Experimentos/ner/CSVs/teste_iob.txt', 'wt') as fileout:
    fileout.writelines(z)

0       AVISO DE ABERTURA DE LICITACAO TOMADA DE PRECO...
1       AVISO DE ABERTURA DE LICITACAO TOMADA DE PRECO...
2       AVISO DE ABERTURA DE LICITACAO PREGAO No 001/2...
3       AVISO DE ABERTURA DE LICITACAO CONVITE No 001/...
4       AVISO DE ABERTURA DE LICITACAO CONVITE No 001/...
                              ...                        
1238    AVISO DE ABERTURA DE LICITACAO PREGAO ELETRONI...
1239    AVISO DE ABERTURA DE LICITACAO PREGAO ELETRONI...
1240    AVISO DE ABERTURA DE LICITACAO SRP PREGAO ELET...
1241    AVISO DE ABERTURA DE LICITACAO PREGAO ELETRONI...
1242    AVISO DE ABERTURA DE LICITACAO PREGAO ELETRONI...
Length: 1243, dtype: object
0       AVISO DE ABERTURA DE LICITACAO TOMADA DE PRECO...
1       AVISO DE ABERTURA DE LICITACAO TOMADA DE PRECO...
2       AVISO DE ABERTURA DE LICITACAO PREGAO No 001/2...
3       AVISO DE ABERTURA DE LICITACAO CONVITE No 001/...
4       AVISO DE ABERTURA DE LICITACAO CONVITE No 001/...
                              ...           

## Create dictionary feature for each word in each sequence in x

In [11]:
def get_features(sentence):
        """Create features for each word in act.
        Create a list of dict of words features to be used in the predictor module.
        Args:
            act (list): List of words in an act.
        Returns:
            A list with a dictionary of features for each of the words.
        """
        sent_features = []
        for i in range(len(sentence)):
            word_feat = {
                'word': sentence[i].lower(),
                'capital_letter': sentence[i][0].isupper(),
                'all_capital': sentence[i].isupper(),
                'isdigit': sentence[i].isdigit(),
                'word_before': sentence[i].lower() if i == 0 else sentence[i-1].lower(),
                'word_after:': sentence[i].lower() if i+1 >= len(sentence) else sentence[i+1].lower(),
                'BOS': i == 0,
                'EOS': i == len(sentence)-1
            }
            sent_features.append(word_feat)
        return sent_features
    
for i in range(len(x)):
    x[i] = get_features(x[i])

## Separate train and test splits (in order)

In [12]:
import math
split_idx = math.floor(0.8*len(x))
x_train = x[0:split_idx]
y_train = y[0:split_idx]
x_test = x[split_idx:]
y_test = y[split_idx:]
len(x_train),  len(x_test), len(x)

(994, 249, 1243)

## Model CRF Trainning

In [13]:
import sklearn_crfsuite
from sklearn_crfsuite import metrics

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=10,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=False,
    all_possible_states=True
)

#crf.fit(x_train, y_train)

try:
    crf.fit(x_train, y_train)
except AttributeError:
    pass


## Evaluation

In [14]:
classes = list(crf.classes_)
classes.remove('O')

y_pred = crf.predict(x_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=classes)

0.6064948842914542

In [15]:
print(metrics.flat_classification_report(
    y_test, y_pred, labels=classes, digits=3))



/home/lucelia_vieira/Experimentos/venv/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass labels=['B-ATO', 'I-ATO', 'B-DATA', 'I-DATA', 'B-PROCESSO', 'B-REDATOR', 'I-REDATOR', 'B-V_ESTIMADO', 'I-V_ESTIMADO', 'B-TIPO', 'I-TIPO', 'B-DISPUTA', 'I-DISPUTA'] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
/home/lucelia_vieira/Experimentos/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

       B-ATO      1.000     1.000     1.000       258
       I-ATO      0.991     0.999     0.995      1023
      B-DATA      0.947     0.570     0.711       158
      I-DATA      0.947     0.784     0.858       459
  B-PROCESSO      0.996     1.000     0.998       241
   B-REDATOR      0.717     0.618     0.663       217
   I-REDATOR      0.950     0.684     0.796       640
B-V_ESTIMADO      0.581     0.090     0.156       277
I-V_ESTIMADO      0.418     0.098     0.159       235
      B-TIPO      0.988     0.884     0.933       190
      I-TIPO      0.936     0.121     0.215      1697
   B-DISPUTA      1.000     0.886     0.939        35
   I-DISPUTA      0.000     0.000     0.000        63

   micro avg      0.944     0.545     0.691      5493
   macro avg      0.806     0.595     0.648      5493
weighted avg      0.898     0.545     0.606      5493



/home/lucelia_vieira/Experimentos/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/lucelia_vieira/Experimentos/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [14]:
#from sklearn.metrics import confusion_matrix
#confusion_matrix(y_test, y_pred, labels=classes)

## Hyperparameter Optimization

In [ ]:
# define fixed parameters and parameters to search
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True,
    all_possible_states=True
)
params_space = {
    'c1': scipy.stats.expon(scale=15.0),
    'c2': scipy.stats.expon(scale=1.0),
}


# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=classes)

# search
rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=50,
                        scoring=f1_scorer)
rs.fit(x_train, y_train)

In [ ]:
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

## Check best estimator on our test data

In [ ]:
sorted_classes = sorted(
    classes,
    key=lambda name: (name[1:], name[0])
)

classes = list(crf.classes_)
classes.remove('O')

y_pred = crf.predict(x_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=classes)

In [ ]:
crf = rs.best_estimator_
y_pred = crf.predict(x_test)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_classes, digits=3
))

## Let’s check what classifier learned

In [ ]:
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

In [ ]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))

print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))

print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])

## Saving the best model

In [ ]:
import joblib

joblib.dump(crf, 'crf_model.pkl')

In [ ]:
model = joblib.load('crf_model.pkl')

In [ ]:
model

In [ ]:
model.classes_